In [0]:
%sql

SELECT DISTINCT SLOT_MNEMONIC FROM 4_prod.raw.mill_sch_appt
WHERE SLOT_MNEMONIC ILIKE '%antenatal%'
LIMIT 100

In [0]:
%sql

SELECT COUNT(DISTINCT a.PERSON_ID)
FROM 4_prod.raw.mill_sch_appt AS a
INNER JOIN 4_prod.raw.mill_sch_event AS e
ON a.SCH_EVENT_ID = e.SCH_EVENT_ID
INNER JOIN 4_prod.raw.mill_person AS p
ON a.PERSON_ID = p.PERSON_ID
WHERE 
  APPT_SYNONYM_FREE ILIKE '%antenatal%'
  AND BEG_DT_TM BETWEEN '2019-12-01' AND '2024-12-31'
  AND BIRTH_DT_TM < '2001-12-01' -- 18 or above on 2019-12-01
LIMIT 100

In [0]:
%sql

WITH ante AS (
SELECT DISTINCT PERSON_ID FROM 4_prod.raw.mill_sch_appt AS a
INNER JOIN 4_prod.raw.mill_sch_event AS e
ON a.SCH_EVENT_ID = e.SCH_EVENT_ID
WHERE 
  APPT_SYNONYM_FREE ILIKE '%antenatal%'
  AND BEG_DT_TM BETWEEN '2019-12-01' AND '2024-12-31'
)
SELECT *
FROM 4_prod.raw.mill_clinical_event AS ce
INNER JOIN ante
ON ce.PERSON_ID = ante.PERSON_ID
WHERE EVENT_TITLE_TEXT ILIKE '%CMV Detected%'
LIMIT 100

In [0]:
%sql

WITH cohort AS (
  SELECT DISTINCT a.PERSON_ID FROM 4_prod.raw.mill_sch_appt AS a
  INNER JOIN 4_prod.raw.mill_sch_event AS e
  ON a.SCH_EVENT_ID = e.SCH_EVENT_ID
  INNER JOIN 4_prod.raw.mill_person AS p
  ON a.PERSON_ID = p.PERSON_ID
  WHERE 
    APPT_SYNONYM_FREE ILIKE '%antenatal%'
    AND BEG_DT_TM BETWEEN '2019-12-01' AND '2024-12-31'
    AND BIRTH_DT_TM < '2001-12-01'
),
ce AS (
  SELECT EVENT_ID, MAX(PERSON_ID) AS PERSON_ID
  FROM 4_prod.raw.mill_clinical_event
  GROUP BY EVENT_ID
)
SELECT b.* 
FROM 4_prod.raw.pi_cde_blob_content AS b
INNER JOIN ce
ON ce.EVENT_ID = b.EVENT_ID
INNER JOIN cohort
ON cohort.PERSON_ID = ce.PERSON_ID
WHERE BLOB_CONTENTS LIKE '%CMV%'

LIMIT 100


In [0]:
%sql

SELECT * FROM 4_prod.raw.mill_person
LIMIT 10